# [KR] BigQuery Hackathon

해당 노트북은 [[KR] BigQuery Hackathon]

## 환경 초기 설정

**중요: 아래 셀에 현재 실습에서 사용하는 Google Cloud 프로젝트 ID를 반드시 입력해야 합니다. 이 값은 실습 환경 내의 모든 리소스에 액세스하기 위한 필수 값입니다. 이 노트북에서 첫 번째로 실행해야 하는 셀입니다.**



In [ ]:
# 이 셀에 프로젝트 아이디를 입력합니다.
PROJECT_ID = 'your-gcp-project' # <-- 여기에 프로젝트 ID를 입력해주세요.

# PROJECT_ID가 공란이 아니도록 확인합니다. 공란일 경우 에러가 발생합니다.
if not PROJECT_ID:
    raise ValueError("ERROR: PROJECT_ID is not set. Please enter your Project ID above.")

print(f"Project ID set to: {PROJECT_ID}")

이제 이 셀을 실행하여 태스크 2 환경을 초기화합니다. 이 셀은 태스크 2에 필요한 라이브러리를 가져오고, BigQuery 클라이언트를 초기화하며, 사용할 주요 변수(테이블 ID 등)를 정의합니다.

In [ ]:
# 태스크 2에 필요한 라이브러리를 가져오고 클라이언트 및 변수를 초기화합니다.
from google.cloud import bigquery
import pandas as pd
import pandas_gbq
from IPython.display import display

# 위의 셀에 PROJECT_ID가 정의되어 있는지 확인
# 이 셀을 실행하기 전, 위에서 프로젝트 ID를 입력하는 셀을 반드시 실행해야 합니다.
if 'PROJECT_ID' not in locals() or not PROJECT_ID:
    raise ValueError("ERROR: PROJECT_ID is not set. Please run the 'Set Your Project ID' cell above first.")

client = bigquery.Client(project=PROJECT_ID, location="us-central1")

DATASET_ID = 'cymbal'

TABLE_ID_REVIEWS = f"{PROJECT_ID}.{DATASET_ID}.multimodal_customer_reviews"
TABLE_ID_CUSTOMERS = f"{PROJECT_ID}.{DATASET_ID}.customers"
table_id_multimodal_reviews = f"{PROJECT_ID}.{DATASET_ID}.multimodal_customer_reviews"
GEMINI_MODEL_NAME = f'{PROJECT_ID}.{DATASET_ID}.gemini_flash_model'
table_id_segment_level_analysis = f"{PROJECT_ID}.{DATASET_ID}.segment_level_gemini_analysis"

print(f"BigQuery Client Initialized for Project ID: {PROJECT_ID}")

def run_bq_query(sql: str, client: bigquery.Client):
    try:
        query_job = client.query(sql)
        print(f"Job {query_job.job_id} in state {query_job.state}")
        if query_job.statement_type == 'SELECT':
            df = query_job.to_dataframe()
            print(f"Query complete. Fetched {len(df)} rows.")
            return df
        else:
            query_job.result()
            print(f"Query for statement type {query_job.statement_type} complete.")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

## 페르소나 프로필 식별
### 고객 데모그래픽 테이블 확인

In [ ]:
%%bigquery
SELECT customer_id, first_name, age, gender, loyalty_member FROM `cymbal.customers` LIMIT 5

### 고객 세그먼트 정의
페르소나 프로필로 사용할 고객 세그먼트의 고유한 조합을 식별하는 쿼리를 작성해주세요.

TABLE_ID_REVIEWS, TABLE_ID_CUSTOMERS를 활용해

* age_group: 40세 미만은 'Younger_Adult', 40세 이상은 'Older_Adult'
* gender_segment: gender 컬럼의 값을 대문자로 변환 (예: 'MALE', 'FEMALE')
* loyalty_status: loyalty_member가 True면 'LOYAL', False이면 'NON_LOYAL'
* text_sentiment: sentiment_json_string 컬럼에서 '$.sentiment'를 추출
* persona_age_group_profile: age_group, gender_segment, loyalty_status 값을 밑줄로 연결하여 하나의 string으로 연결 (ex. "Older_Adult_FEMALE_LOYAL"과 같은 형태)

위 칼럼들을 새로 생성합니다.

In [ ]:
# cymbal.multimodal_customer_reviews 테이블과 cymbal.customers 테이블을 조인하고
# age_group, gender_segment, loyalty_status, text_sentiment, persona_age_group_profile 컬럼을 생성합니다. 
# 위 컬럼을 모두 포함하는 cymbal.unique_segment_profiles 테이블을 생성하는 코드를 작성합니다.

sql_data_for_viz = f"""
# YOUR SQL CODE FROM HERE
"""

In [ ]:
%%bigquery
SELECT * FROM `cymbal.unique_segment_profiles` LIMIT 5

## 페르소나 데이터 시각화를 통한 EDA
이 예시 쿼리는 각 세그먼트의 고객 수를 집계하고 해당 데이터를 막대 차트로 시각화합니다. 
자유롭게 EDA를 위한 코드를 추가해보세요.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def plot_customer_distribution(df: pd.DataFrame):
    if df is None or df.empty:
        print("DataFrame is empty. Skipping plot generation.")
        return

    sns.set_theme(style="whitegrid", font_scale=1.0)

    plt.figure(figsize=(10, 6))

    palette = sns.cubehelix_palette(n_colors=len(df), start=.5, rot=-.75, dark=0.3, light=0.7)

    ax = sns.barplot(
        x='persona_age_group_profile', y='customer_count', data=df,
        palette=palette, hue='persona_age_group_profile', legend=False
    )

    for p in ax.patches:
        ax.annotate(f'{int(p.get_height()):,}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 8), textcoords='offset points',
                    fontsize=9, color='dimgray')

    ax.set(title='Customer Segment Distribution', xlabel='Persona Profile', ylabel='Number of Customers')
    ax.title.set_size(16); ax.title.set_weight('bold'); ax.xaxis.label.set_size(12);
    ax.yaxis.label.set_size(12); ax.title.set_position([.5, 1.05]);

    plt.xticks(rotation=45, ha='right')
    plt.ylim(0, df['customer_count'].max() * 1.15)
    sns.despine()
    plt.tight_layout()
    plt.show()

In [ ]:
print("Querying data for visualization...")
df_for_viz = run_bq_query(sql_data_for_viz, client)

print("Generating plot...")
plot_customer_distribution(df_for_viz)

## 추가 시각화
### 다음 코드 블럭을 활용해 자유롭게 EDA를 수행해주세요.

### 예시 주제
주제 1: 연령대와 충성도 간의 관계 분석
age_group과 loyalty_status의 조합이 어떻게 고객 수에 영향을 미치는지 시각화하여, 특정 연령대의 고객이 더 충성도가 높은 경향이 있는지 등을 분석합니다.

주제 2: 세그먼트별 지리적 분포 시각화
address_city 정보를 활용하여 각 persona_age_group_profile별 고객들이 특정 도시에 집중되어 있는지 또는 넓게 분포되어 있는지 시각화하여 지리적 특성을 파악합니다.

In [ ]:
# YOUR CODE FROM HERE

## Gemini를 사용한 상세 페르소나 생성
주어진 값을 바탕으로 상세 페르소나를 생성하는 코드를 작성합니다.
이 섹션의 일부 코드 셀은 이미 완성되어 있습니다. 제공된 코드를 바탕으로 비어있는 부분을 직접 작성해 코드를 완성합니다.

In [ ]:
FINAL_DESTINATION_TABLE_ID = "segment_level_gemini_analysis"
table_id_segment_level_analysis = f"{DATASET_ID}.{FINAL_DESTINATION_TABLE_ID}"

gemini_prompt_template = """
고객 세그먼트 프로필 "{p}"를 기반으로 유효한 단일 JSON 객체를 생성합니다.
JSON은 다음 키를 반드시 포함해야 합니다:
"persona_description" (이 페르소나에 대한 간결한 한 문장 요약),
"summary" (그들의 예상되는 선호도에 대한 더 자세한 요약),
"motivations" (구매 결정에 영향을 미치는 요인),
"needs" (제품 또는 서비스에서 찾는 것),
"marketing_pitch" (그들을 타겟팅하는 짧은 마케팅 문구).
전체 출력은 하나의 JSON 객체여야 하며, 텍스트 본문은 한국어로 구성합니다.
"""

In [ ]:
TEMP_TABLE_ID = "temp_gemini_prompts"
TEMP_TABLE = f"{PROJECT_ID}.{DATASET_ID}.{TEMP_TABLE_ID}"

print("Loading unique persona profiles from BigQuery.")
sql_load_profiles = f"SELECT persona_age_group_profile FROM `{DATASET_ID}.unique_segment_profiles` ORDER BY 1"
source_df = run_bq_query(sql_load_profiles, client)

if source_df is not None and not source_df.empty:
    unique_profiles_df = source_df['persona_age_group_profile'].astype(str).drop_duplicates().to_frame(name='profile_name')
    print(f"Found {len(source_df)} total rows. Analyzing {len(unique_profiles_df)} unique profiles. Preparing for batch analysis...")
    prompts_df = pd.DataFrame({
        'profile_name': unique_profiles_df['profile_name'],
        'prompt': unique_profiles_df['profile_name'].apply(
            lambda p: gemini_prompt_template.format(p=p)
        )
    })

    pandas_gbq.to_gbq(
        prompts_df,
        f'{DATASET_ID}.{TEMP_TABLE_ID}',
        project_id=PROJECT_ID,
        if_exists='replace',
        location='us-central1'
    )
    print("✅ Temporary prompts table created successfully.")
else:
    print("No profiles found to analyze. Please ensure 'unique_segment_profiles' table is correctly populated.")

### Gemini 배치 분석 수행
cymbal.temp_gemini_prompts 테이블에 저장된 프롬프트를 사용하여 BigQuery ML의 ML.GENERATE_TEXT 함수를 호출하는 SQL 쿼리를 작성합니다.

**모델 이름**: GEMINI_MODEL_NAME 변수를 사용합니다.\
**입력 테이블**: cymbal.temp_gemini_prompts 테이블을 입력값으로 사용합니다.\
**STRUCT 옵션**: 이 부분은 제공되니 활용하여 쿼리를 완성하세요.\
**결과 컬럼**: Gemini 응답 (ml_generate_text_llm_result)과 함께 profile_name 컬럼을 포함하도록 쿼리를 작성합니다.

In [ ]:
# ML.GENERATE_TEXT를 호출하는 SQL 쿼리를 작성합니다.

sql_batch_analysis = f"""
    -- 여기에 ML.GENERATE_TEXT를 호출하는 SQL 쿼리를 작성하세요.
    -- cymbal.temp_gemini_prompts 테이블을 입력값으로 설정하고,
    -- GEMINI_MODEL_NAME 모델과 아래 STRUCT 옵션을 사용합니다.
    -- 결과에 profile_name과 Gemini의 분석 결과를 포함해야 합니다.
    SELECT
        t2.profile_name,
        t1.ml_generate_text_llm_result AS analysis
    FROM
        ML.GENERATE_TEXT(
            MODEL `{GEMINI_MODEL_NAME}`,
            (SELECT * FROM `{TEMP_TABLE}`), -- TEMP_TABLE은 cymbal.temp_gemini_prompts를 가리킵니다.
            STRUCT(0.5 AS temperature, 1024 AS max_output_tokens, TRUE AS flatten_json_output)
        ) AS t1
    JOIN
        `{TEMP_TABLE}` AS t2
    ON
        t1.prompt = t2.prompt;
"""

print("\nStarting single batch analysis using Gemini on BigQuery...")
# Execute your SQL query
df_all_analysis = run_bq_query(sql_batch_analysis, client)

if df_all_analysis is not None:
    print("✅ Analysis complete.")
else:
    print("❌ Gemini analysis failed or returned no results.")

### 분석 결과 저장

In [ ]:
if df_all_analysis is not None and not df_all_analysis.empty:
    print(f"\n Saving {len(df_all_analysis)} analyses to BigQuery table: {table_id_segment_level_analysis}")
    pandas_gbq.to_gbq(
        df_all_analysis,
        table_id_segment_level_analysis,
        project_id=PROJECT_ID,
        if_exists='replace',
        location='us-central1'
    )
    print("✅ Results successfully saved to BigQuery.")
else:
    print("No analysis results to save. Skipping table creation.")

## 생성된 페르소나 분석 테이블 확인
### Gemini로 분석한 원본 테이블의 내용을 표시하여 확인합니다.

In [ ]:
df_raw_analysis = run_bq_query(f"SELECT * FROM `{table_id_segment_level_analysis}` LIMIT 5", client)
if df_raw_analysis is not None:
    with pd.option_context('display.max_colwidth', None):
        display(df_raw_analysis)

## 최종 인사이트 및 페르소나 테이블 생성

In [ ]:
table_id_multimodal_reviews = f"{PROJECT_ID}.{DATASET_ID}.multimodal_customer_reviews"
TABLE_ID_CUSTOMERS = f"{PROJECT_ID}.{DATASET_ID}.customers"
table_id_segment_level_analysis = f"{PROJECT_ID}.{DATASET_ID}.segment_level_gemini_analysis"

table_id_final_customer_insights = f"{PROJECT_ID}.{DATASET_ID}.final_customer_insights"
sql_create_final_table = f"""
CREATE OR REPLACE TABLE `{table_id_final_customer_insights}` AS
WITH EnrichedData AS (
    SELECT mcr.*, c.first_name, c.last_name, c.age, c.gender, c.loyalty_member,
        CONCAT(
            CASE WHEN c.age < 40 THEN 'Younger_Adult' ELSE 'Older_Adult' END, '_',
            UPPER(c.gender), IF(c.loyalty_member, '_LOYAL', '_NON_LOYAL')
        ) AS persona_age_group_profile
    FROM `{table_id_multimodal_reviews}` AS mcr
    JOIN `{TABLE_ID_CUSTOMERS}` AS c ON mcr.customer_id = c.customer_id
)
SELECT enriched.*, persona.analysis AS gemini_persona_analysis
FROM EnrichedData enriched
LEFT JOIN `{table_id_segment_level_analysis}` persona ON enriched.persona_age_group_profile = persona.profile_name;
"""

print(f"1. Creating the final customer insights table '{table_id_final_customer_insights}'...")
run_bq_query(sql_create_final_table, client)
print("✅ Final customer insights table created successfully.")

final_persona_table_id = f"{PROJECT_ID}.{DATASET_ID}.customer_persona_definitions"
sql_create_personas = f"""
CREATE OR REPLACE TABLE `{final_persona_table_id}` AS
WITH cleaned_analysis AS (
  SELECT
    profile_name AS profile,
    -- Clean the JSON string by removing markdown backticks and whitespace
    TRIM(REGEXP_REPLACE(analysis, r'(?i)(^```json\\s*|\\s*```$)', '')) as cleaned_json
  FROM
    `{table_id_segment_level_analysis}`
)
SELECT
    profile AS persona_age_group_profile,
    JSON_EXTRACT_SCALAR(cleaned_json, '$.persona_description') AS persona_segment_description
FROM
    cleaned_analysis
WHERE
    JSON_EXTRACT_SCALAR(cleaned_json, '$.persona_description') IS NOT NULL;
"""
print(f"\n2. Creating final persona definitions table from Gemini output: {final_persona_table_id}...")
run_bq_query(sql_create_personas, client)
print("✅ Final persona definitions table created successfully.")

print(f"\n--- 3. Verifying Final Customer Persona Definitions (Generated by Gemini) ---")
df_personas = run_bq_query(f"SELECT * FROM `{final_persona_table_id}` ORDER BY 1", client)
if df_personas is not None:
    with pd.option_context('display.max_colwidth', None):
        display(df_personas)
